In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle 
from queue import PriorityQueue
from scipy.spatial.distance import cosine
import gradio as gr

from IPython.display import clear_output
clear_output(wait=False)

In [20]:
with open('output/anime_df.pkl', 'rb') as f:
    anime_df = pickle.load(f)

with open('output/user_df.pkl', 'rb') as f:
    user_df = pickle.load(f)

with open('output/anime_name_dict.pkl', 'rb') as f:
    anime_dict = pickle.load(f)

In [57]:
# def find_closest_by_intersection(df, new_list):
#     #max_intersection_size = 0.6
#     user_list=[]

#     new_set = set(new_list)

#     for _, row in df.iterrows():
#         row_set = set(row['anime_id'])
#         intersection_size = len(new_set.intersection(row_set))/len(new_set)


#         user_que = PriorityQueue(maxsize=500)
#         if user_que.full():
#             user_que.get()
#         user_que.put(((-1*intersection_size),row['user_id']))
    
#     print(user_que.get())
    
#     while user_que.empty==False:
#         user_list.append(user_que.get()[1])

#     return user_list

In [ ]:
def find_closest_by_intersection(df, new_list):
    max_intersection_size = 0.6
    user_list=[]

    new_set = set(new_list)

    for _, row in df.iterrows():
        row_set = set(row['anime_id'])
        intersection_size = len(new_set.intersection(row_set))/len(new_set)


        if intersection_size > max_intersection_size:
            user_list.append(row['user_id'])

    return user_list

In [22]:
def cosine_similarity(vec1, vec2):
    return 1-cosine(vec1, vec2)

In [41]:
def find_top_matches(new_user,anime_df= anime_df,user_df= user_df,anime_dict= anime_dict):
    new_user_anime_ids = [anime_dict[name] for name in new_user]
    closest_users = set(find_closest_by_intersection(user_df, new_user_anime_ids))
    anime_list = set(anime_df['MAL_ID'])
    anime_set = set()

    for _,row in user_df.iterrows():
        if row['user_id'] in  closest_users:
            anime_ids = row['anime_id']
            for anime in anime_ids:
                if anime in anime_list:
                    anime_set.add(anime)
    
    embeddings_to_search = anime_df[anime_df['MAL_ID'].isin(anime_set)]
    embeddings_to_search = embeddings_to_search[~embeddings_to_search['MAL_ID'].isin(new_user_anime_ids)]
    embeddings_to_search['anime_embedding'] = embeddings_to_search['anime_embedding'].apply(lambda x: [float(i) for i in x])

    anime_df['anime_embedding'] = anime_df['anime_embedding'].apply(lambda x: [float(i) for i in x])
    new_user_set = set(new_user_anime_ids)
    new_user_embed=[]
    for _,row in anime_df.iterrows():
        if row['MAL_ID'] in new_user_set:
            new_user_embed.append(row['anime_embedding'])

    new_user_embed = np.array(new_user_embed)
    new_user_embed = new_user_embed.mean(axis=0)

    que = PriorityQueue(maxsize=20)
    for _,row in embeddings_to_search.iterrows():
        similarity = cosine_similarity(row['anime_embedding'],new_user_embed)
        if que.full():
            que.get()
        que.put((similarity,row['Name']))
    
    return que

In [47]:
dropdown_choices=anime_df['Name'].tolist()

def process_choice(choice):
    ans = find_top_matches(choice)
    suggestions=[]
    while ans.empty() == False:
        suggestions.append(ans.get()[1])
    return f"{suggestions}"

demo = gr.Interface(
    fn=process_choice,
    inputs=gr.Dropdown(choices=dropdown_choices, multiselect=True, label='Select your Watched animes'),
    outputs=gr.Textbox(label='Output'),
)

demo.launch()


Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.
